In [1]:
import numpy as np
import pandas as pd

import dask.dataframe as dd
import dask.array as da
import dask.bag as db

sevn_output_Z0.001A1L1/sevn_output_Z0.001A1L1/0/evolved_0.dat

In [2]:
output = dd.read_csv('sevn_output_Z0.001A1L1/sevn_output_Z0.001A1L1/0/output_*.csv')
evolved = dd.read_table('sevn_output_Z0.001A1L1/sevn_output_Z0.001A1L1/0/evolved_*.dat', sep='\s+')
logfile = dd.read_table('sevn_output_Z0.001A1L1/sevn_output_Z0.001A1L1/0/logfile_*.dat', sep=';', header=None, names=['Object', 'name', 'ID', 'event', 'time', 'info'])

print(f'Number of pratitions for evolved files : {evolved.npartitions}')
print(f'Number of pratitions for output files : {output.npartitions}')
print(f'Number of pratitions for logfile files : {logfile.npartitions}')

Number of pratitions for evolved files : 30
Number of pratitions for output files : 90
Number of pratitions for logfile files : 30


Filtering output files to get only binaries black holes

In [3]:
idxBHBH=(output.RemnantType_0==6) & (output.RemnantType_1==6) & (output.Semimajor.notnull())
output_bhbh=output[idxBHBH] #Distributed DataFrame containing only binaries BB
# output_bhbh = output_bhbh.compute()

Filtering logfile files to count the number of RL overflows and CE

In [4]:
test_logfile = logfile.partitions[0].compute()

In [5]:
test_logfile[(test_logfile['Object']=='B') & (test_logfile['event']=='RLO_BEGIN')].groupby('name').count(3)

TypeError: GroupBy.count() takes 1 positional argument but 2 were given

In [ ]:
test_logfile[(test_logfile['Object']=='B') & (test_logfile['event']=='RLO_BEGIN')].groupby('name').event.count().to_frame(name='RLO_BEGIN')

In [ ]:
#NON RANNARLO, e' un reperto storico
# logfile[(logfile['event']=='RLO_BEGIN')].groupby('name').event.value_counts().compute()
logfile[(logfile['event']=='RLO_BEGIN')].groupby('name').count().compute()

In [ ]:
bhbh = output_bhbh.merge(evolved, on='name', how='inner')
# bhbh.head()